In [1]:
#change current path to "../../"
import os
import sys
sys.path.append("../../")

In [2]:
from Model.Environment import Game, GameObserver, Environment, TransitionProfile
import ipywidgets as widgets
from IPython.display import display

In [3]:
game = Game(3, [2, 3, 3])

In [4]:
env = Environment(2, 2)

In [5]:
print(game)

Game: [2, 3, 3] 
 PossibileActions:[2, 3, 3] 
 NPlayers: 3 
 Transition: ['{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}'] 
 Payoff: [[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


In [6]:
game.setNPlayers(2)
print(game)
game.setNPlayers(3)
print(game)
game.setTransition((0, 0, 0), 1, 0.5)

Game: [2, 3] 
 PossibileActions:[2, 3] 
 NPlayers: 2 
 Transition: ['{}', '{}', '{}', '{}', '{}', '{}'] 
 Payoff: [[[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]]
Game: [2, 3, 1] 
 PossibileActions:[2, 3, 1] 
 NPlayers: 3 
 Transition: ['{}', '{}', '{}', '{}', '{}', '{}'] 
 Payoff: [[[[0. 0. 0.]]

  [[0. 0. 0.]]

  [[0. 0. 0.]]]


 [[[0. 0. 0.]]

  [[0. 0. 0.]]

  [[0. 0. 0.]]]]
{1: 0.5}


In [7]:
nPlayersWidget = widgets.IntSlider(
    value=game.NPlayers,
    min=2,
    max=4,
    step=1,
    description='Number of players:',
)

In [8]:
nPlayersWidget.observe(lambda change: game.setNPlayers(change.new), names='value')

In [9]:
display(nPlayersWidget)

IntSlider(value=3, description='Number of players:', max=4, min=2)

In [10]:
class ActionProfileWidget(GameObserver):
    def __init__(self, game, on_change=None):
        self.game = game
        self.on_change = on_change
        self.box = widgets.HBox()
        self.widgets = []

        game.attach(self)

        self.update(game)
            
        display(self.box)
        
    def getWidget(self):
        return self.box
    
    def get(self):
        return tuple([w.value for w in self.widgets])
    
    def update(self, game):
        if(game.NPlayers < len(self.widgets)):
            self.box.children = self.box.children[:game.NPlayers]
            self.widgets = self.widgets[:game.NPlayers]
        else:
            for i in range(min(game.NPlayers, len(self.widgets))):
                self.widgets[i].max = game.possibleActions[i]-1

            for i in range(max(0, game.NPlayers - len(self.widgets))):
                newWidget = widgets.IntSlider(
                    value=0,
                    min=0,
                    max=game.possibleActions[len(self.widgets)]-1,
                    step=1,
                    description='Player '+str(len(self.widgets))+':',
                )
                self.widgets.append(newWidget)
                newWidget.observe(self.on_change, names='value')

            self.box.children = self.widgets

    def setOnChange(self, on_change):
        self.on_change = on_change
        for w in self.widgets:
            w.observe(on_change, names='value')
        


In [11]:
display(nPlayersWidget)

IntSlider(value=3, description='Number of players:', max=4, min=2)

In [12]:
actionProfileWidget = ActionProfileWidget(game, lambda x : print(actionProfileWidget.get()))

(1, 0, 0)
(0, 0, 0)
(1, 0, 0)
(0, 0, 0)
(1, 0, 0)
(0, 0, 0)
(1, 0, 0)
(0, 0, 0)


In [21]:
class TransitionProbabilityWidget(GameObserver):
    def __init__(self, game, NGames = 2, actionProfile = (0, 0), on_change=None):
        self.game = game
        self.NGames = NGames
        self.on_change = on_change
        self.box = widgets.HBox()
        self.widgets = []
        self.actionProfile = actionProfile

        game.attach(self)

        self.update(game)
            
        display(self.box)
        
    def getWidget(self):
        return self.box
    
    def get(self):
        return tuple([w.value for w in self.widgets])
    
    def update(self, game):
        if(self.NGames < len(self.widgets)):
            self.box.children = self.box.children[:game.NGames]
            self.widgets = self.widgets[:game.NGames]
        else:
            nexts, ps = game.getTransition(self.actionProfile).getTransitions()

            for i in range(max(0, self.NGames - len(self.widgets))):
                val = 0
                if(len(self.widgets) in nexts):
                    val = game.getTransition(self.actionProfile).getProbability(len(self.widgets))

                newWidget = widgets.BoundedFloatText(
                    value=val,
                    min=0,
                    max=1,
                    step=0.01,
                    description='Game '+str(len(self.widgets))+':',
                )
                self.widgets.append(newWidget)
                newWidget.observe(lambda change : 
                                    self.setTransitionProbs(),
                                    names='value')
                
            for i in range(min(self.NGames, len(self.widgets))):
                if i not in nexts:
                    self.widgets[i].value = 0

            for n, p  in zip(nexts, ps):
                self.widgets[n].value = p

            self.box.children = self.widgets

    def setTransitionProbs(self):
        tp = TransitionProfile({})

        for i, w in enumerate(self.widgets):
            if(w.value > 0):
                tp.setTransition(i, w.value)

        self.game.setTransitionProfile(self.actionProfile, tp)


    def updateActionProfile(self, actionProfile):
        self.actionProfile = actionProfile
        transitionProfile = game.getTransition(tuple(actionProfile))
        nextGames, porobabilities = transitionProfile.getTransitions()

        for (i, w) in enumerate(self.widgets):
            if(i not in nextGames):
                w.value = 0
        
        for g, p in zip(nextGames, porobabilities):
            self.widgets[g].value = p

In [22]:
transitionProbabilityWidget = TransitionProbabilityWidget(game, 2, actionProfileWidget.get(), lambda p, next : game.setTransition(actionProfileWidget.get(), next, p))

In [15]:
actionProfileWidget.setOnChange(lambda x : transitionProbabilityWidget.updateActionProfile(actionProfileWidget.get()))

In [16]:
display(actionProfileWidget.getWidget())

In [17]:
class GameEditor:
    nPlayers = 1
    def __init__(self, nPlayers:int=1):
        self.nPlayers = nPlayers


    def getWidget(self):
        return self.nPlayers